In [2]:
import numpy as np
from trajectory import Trajectory, generate_trajectory, generate_trajectories, stochastic_policy_adapter
from solver import value_iteration, stochastic_policy_from_value_expectation
from snake_ladder import SnakeLadderWorld
import tensorflow as tf
import datetime

In [3]:
# define some consants
world_size = 20
shortcut_density = 0.1
success_prob = .9
n_trajectories_per_policy = 5000

In [4]:
# create our world
world = SnakeLadderWorld(size=world_size, shortcut_density=shortcut_density)

# set up the reward function
reward = np.zeros(world.n_states)
reward[-1] = 1.0
# start state
start = [0]

# set up terminal states
terminal = [world.size - 1]

world.game_board

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 18, 14,  4, 16,
       17, 18, 19])

In [5]:
# create some policies
# create "fixed" policies which each execeute one of the three actions w/ prob p (success_prob)
# randomly sample from all actions w/ prob 1 - p
# so excute one action with prob p + 1/3(1 - p) and others with 1/3(1 -  p)
policies_fixed = []

for i in range(3):
    def policy(state, action = i):
        if success_prob >= np.random.uniform():
            return action
        else:
            return np.random.choice(3)
    policies_fixed.append(policy)

In [6]:
# verify our fixed policies
samples = 10000
means = [0,0,0]
for i in range(samples):
    for j in range(3):
        means[j] += policies_fixed[j](None)
means = np.array(means) / samples
means

array([0.1029, 0.9988, 1.9074])

In [9]:
# generate an "optimal" policy w/ value iteration
discount = .7
weighting = lambda x: x

value = value_iteration(world.p_transition, reward, discount)
policy = stochastic_policy_from_value_expectation(world, value)
policy_exec = stochastic_policy_adapter(policy)

policy

array([[0.28683839, 0.32637941, 0.3867822 ],
       [0.2886961 , 0.32320423, 0.38809967],
       [0.28739578, 0.32042988, 0.39217434],
       [0.28736003, 0.32051728, 0.39212269],
       [0.28636628, 0.32182627, 0.39180744],
       [0.2840794 , 0.32506086, 0.39085974],
       [0.28189816, 0.33028897, 0.38781287],
       [0.27836278, 0.34017374, 0.38146348],
       [0.28195326, 0.34214716, 0.37589959],
       [0.2913629 , 0.32035959, 0.38827751],
       [0.29342145, 0.31593565, 0.3906429 ],
       [0.29674026, 0.30892015, 0.39433959],
       [0.31866611, 0.29159184, 0.38974205],
       [0.2937735 , 0.30513304, 0.40109346],
       [0.18462666, 0.35876585, 0.45660748],
       [0.27562207, 0.3371599 , 0.38721803],
       [0.27005555, 0.3472759 , 0.38266855],
       [0.26410717, 0.35850648, 0.37738635],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

In [16]:
# create list of policies
#policies = policies_fixed
policies = []
policies.append(policy_exec) # add expert policy to list
policies.append(world._smartish_policy)

In [17]:
# generate trajectories with policies
trajectories_list = []
for i, policy in enumerate(policies):
    trajectories = list(generate_trajectories(n_trajectories_per_policy, world, policies[i], start, terminal))
    trajectories = [t._t for t in trajectories]
    trajectories_list.append(trajectories)

In [18]:
# a trajectory from policy 0
trajectories_list[0][0]

[(0, 2, 9), (9, 1, 13), (13, 2, 19)]

In [19]:
# create list of all trajectories, each is a 2d list

x_data = []
for i, policy in enumerate(policies):
    x_data.extend([np.matrix(t).tolist() for t in trajectories_list[i]])

In [20]:
# label trajectories

y_data = []
for i in range(len(policies)):
    y_data.extend([i] * n_trajectories_per_policy)

In [21]:
# shuffle x,y data together
import random
temp = list(zip(x_data,y_data))
random.shuffle(temp)
x_data, y_data = zip(*temp)

In [22]:
# convert trajectories to ragged tensors
x_data = tf.ragged.constant(x_data)
max_seq = int(x_data.bounding_shape()[-2])
print(max_seq)
x_data.bounding_shape()

29


<tf.Tensor: shape=(3,), dtype=int64, numpy=array([10000,    29,     3])>

In [23]:
# convert y to numpy array
y_data = np.array(y_data)
y_data

array([1, 0, 0, ..., 1, 0, 0])

In [24]:
# split into train and test
test_prop = .20
test_n = int(len(y_data) * test_prop)
x_test = x_data[:test_n, :, :]
y_test = y_data[:test_n]
x_train = x_data[test_n:,:,:]
y_train = y_data[test_n:]
x_test.shape, x_train.shape, y_test.shape, y_train.shape

(TensorShape([2000, None, None]),
 TensorShape([8000, None, None]),
 (2000,),
 (8000,))

In [25]:
# create LSTM model
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_seq,3), dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [26]:
metrics = ['accuracy']

lstm_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=metrics)
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                17408     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 21,828
Trainable params: 21,828
Non-trainable params: 0
_________________________________________________________________


In [29]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


lstm_model.fit(x_train,y_train, epochs=100, batch_size=int(n_trajectories_per_policy / 10), validation_data=(x_test,y_test), callbacks=[tensorboard_callback])

Epoch 1/100
16/16 [==============================] - 1s 44ms/step - loss: 0.8978 - accuracy: 0.8481 - val_loss: 0.8915 - val_accuracy: 0.8505
Epoch 2/100
16/16 [==============================] - 1s 38ms/step - loss: 0.8987 - accuracy: 0.8482 - val_loss: 0.8930 - val_accuracy: 0.8475
Epoch 3/100
16/16 [==============================] - 1s 38ms/step - loss: 0.8995 - accuracy: 0.8479 - val_loss: 0.8896 - val_accuracy: 0.8530
Epoch 4/100
16/16 [==============================] - 1s 38ms/step - loss: 0.8990 - accuracy: 0.8456 - val_loss: 0.8882 - val_accuracy: 0.8545
Epoch 5/100
16/16 [==============================] - 1s 39ms/step - loss: 0.8980 - accuracy: 0.8469 - val_loss: 0.8880 - val_accuracy: 0.8545
Epoch 6/100
16/16 [==============================] - 1s 37ms/step - loss: 0.8960 - accuracy: 0.8500 - val_loss: 0.8880 - val_accuracy: 0.8525
Epoch 7/100
16/16 [==============================] - 1s 39ms/step - loss: 0.8948 - accuracy: 0.8508 - val_loss: 0.8862 - val_accuracy: 0.8545
Epoch 

In [30]:
from sklearn.metrics import confusion_matrix

y_predicted = lstm_model.predict_classes(x_test)

# get confusion matrix
cm = confusion_matrix(y_test, y_predicted)

# normalize diagonal entries
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# accuracy by class (indexes 0-2 refres to fixed policies)
# index 3 refres to expert policy
cm.diagonal()

array([0.93493493, 0.84715285])